In [ ]:
import urllib
from urllib.request import urlopen, urlretrieve
from bs4 import BeautifulSoup
import regex as re
from tqdm.notebook import tqdm
import os

site = 'http://www.lada-granta.net/'
folder_names = [
    ['0', 'Отзывы', 'f=139', 'LADA GRANTA'],
    ['1', 'Двигатель', 'f=22', 'LADA GRANTA'],
    ['2', 'Трансмиссия', 'f=23', 'LADA GRANTA'],
    ['3', 'Ходовая', 'f=24', 'LADA GRANTA'],
    ['4', 'Колеса', 'f=167', 'LADA GRANTA'],
    ['5', 'Электрооборудование', 'f=25', 'LADA GRANTA'],
    ['6', 'Мультимедиа', 'f=202', 'LADA GRANTA'],
    ['7', 'Кузов', 'f=26', 'LADA GRANTA'],
    ['8', 'Жидкости', 'f=27', 'LADA GRANTA'],
    ['9', 'Сервис', 'f=20', 'LADA GRANTA'],
    ['10', 'Эксплуатация', 'f=21', 'LADA GRANTA'],
    ['11', 'Тюнинг', 'f=28', 'LADA GRANTA'],
    ['12', 'Электрооборудование-Омыватели', 'f=184', 'LADA GRANTA'],
    ['13', 'Электрооборудование-Зажигание', 'f=198', 'LADA GRANTA'],
    ['14', 'Электрооборудование-Датчики', 'f=199', 'LADA GRANTA'],
    ['15', 'Электрооборудование-Проводка', 'f=200', 'LADA GRANTA'],
    ['16', 'Электрооборудование-Обогрев', 'f=201', 'LADA GRANTA'],
    ['17', 'Электрооборудование-Фары', 'f=185', 'LADA GRANTA'],
    ['18', 'Двигатель-Зажигание', 'f=195', 'LADA GRANTA'],
    ['19', 'Двигатель-Выпуск', 'f=194', 'LADA GRANTA'],
    ['20', 'Двигатель-Зажигание', 'f=195', 'LADA GRANTA'],
    ['21', 'Двигатель-Охлаждение', 'f=193', 'LADA GRANTA'],
    ['22', 'Двигатель-ГРМ', 'f=197', 'LADA GRANTA'],
    ['24', 'Двигатель-Смазка', 'f=196', 'LADA GRANTA'],
    ['25', 'Трансмиссия-МКПП', 'f=168', 'LADA GRANTA'],
    ['26', 'Трансмиссия-АКПП', 'f=169', 'LADA GRANTA'],
    ['27', 'Трансмиссия-АМТ', 'f=170', 'LADA GRANTA'],
    ['28', 'Ходовая-Подвеска', 'f=166', 'LADA GRANTA'],
    ['29', 'Ходовая-Тормозная', 'f=165', 'LADA GRANTA'],
    ['30', 'Ходовая-Рулевая', 'f=164', 'LADA GRANTA'],
    ['31', 'Кузов-Общее', 'f=176', 'LADA GRANTA'],
    ['32', 'Кузов-Отопление', 'f=174', 'LADA GRANTA'],
    ['33', 'Кузов-Кузов', 'f=175', 'LADA GRANTA'],
    ['34', 'Кузов-Салон', 'f=173', 'LADA GRANTA'],
    ['35', 'Жидкости-Топливо', 'f=182', 'LADA GRANTA'],
    ['36', 'Жидкости-Технические', 'f=180', 'LADA GRANTA'],
    ['37', 'Жидкости-ТрМасла', 'f=179', 'LADA GRANTA'],
    ['38', 'Жидкости-Смазки', 'f=177', 'LADA GRANTA'],
    ['39', 'Жидкости-МоМасла', 'f=178', 'LADA GRANTA'],
    ['40', 'Жидкости-Присадки', 'f=181', 'LADA GRANTA'],
    ['41', 'Жидкости-Автохимия', 'f=183', 'LADA GRANTA'],
    ['42', 'Тюнинг-Стайлинг', 'f=142', 'LADA GRANTA'],
    ['43', 'Тюнинг-Тюнинг', 'f=143', 'LADA GRANTA'],
    ['44', 'Тюнинг-Противоугон', 'f=141', 'LADA GRANTA'],
    ['45', 'Тюнинг-Автозвук', 'f=140', 'LADA GRANTA']
]

На странице раздела у нас две задачи:  
1. Найти и сохранить все ссылки на темы
2. Найти ссылку на следующую страницу раздела или убедиться, что это последняя страница раздела.

In [ ]:
def collect_themes(site, folder):
    
    site_address = site + 'forumdisplay.php?' + folder
    themes = []
    flag = True

    while flag:
    
        resp = urlopen(site_address) # скачиваем файл
        html = resp.read().decode('utf-8') # считываем содержимое
        soup = BeautifulSoup(html, 'html.parser') # делаем суп
    
        flag = False
        for link in soup.find_all('a'):
            if link.has_attr('href') and link.has_attr('id'):
                if re.search('thread_title_', str(link.get('id'))):
                    s = link.get('href')
                    s = site + 'showthread.php?' + re.search('t=[0-9]+', s)[0]
                    themes.append([link.get_text(), s])
            if not flag and link.has_attr('href') and link.has_attr('rel'):
                if link.get('rel')[0] == 'next':
                    flag = True
                    s = link.get('href')
                    site_address = site + 'forumdisplay.php?' + re.search(folder + '[\S]+', s)[0]
    return themes

На странице темы у нас три задачи:  
1. Понять, что являеся шапкой темы (это сообщение #1), прочитать ее только один раз - НЕ СДЕЛАНО
2. Прочитать и сохранить все сообщения (кроме шапки) на странице темы
3. Найти ссылку на следующую страницу темы или убедиться, что это последняя страница темы.

In [ ]:
def collect_messages(site, themes):

    messages = []
    for theme in tqdm(themes):
        
        print(theme[0])
        theme_address = theme[1]
        thread = re.search('t=[0-9]+', theme_address)[0]
        flag = True
        theme_messages = []
        post_data = {}
        
        while flag:
            resp = urlopen(theme_address) # скачиваем файл
            html = resp.read().decode('utf-8', errors='ignore') # считываем содержимое
            soup = BeautifulSoup(html, 'html.parser') # делаем суп
            #Собираем некоторые метаданные сообщений на текущей странице
            tables = soup.find_all('table')
            for next_table in tables:
                if next_table.has_attr('id') and re.search('post[0-9]+', next_table['id']):
                    post_column = next_table.find('td')
                    post_date = post_column.get_text().strip()
                    post_data[re.sub('post', '', next_table['id'])] = [post_date]
            #Собираем сами сообщения
            divs = soup.find_all('div')
            for div in divs:
                if div.has_attr('id') and re.search('(postmenu_)([0-9]+)(_menu)', div['id']):
                    continue
                if div.has_attr('id') and re.search('postmenu_', div['id']):
                    author = div.find('a')
                    if author != None:
                        author_id = re.search('u=[0-9]+', author['href'])[0]
                        author_nickname = author.get_text()
                    else:
                        author_id = div.get_text().strip()
                        author_nickname = 'Guest'
                    post_data[re.sub('postmenu_', '', div['id'])].append([author_id, author_nickname])
                if div.has_attr('id') and re.search('post_message_', div['id']):
                    message_id = re.sub('post_message_', '', div['id'])
                    if message_id in post_data:
                        post_date = post_data[message_id][0]
                        author = post_data[message_id][1][1]
                    else:
                        post_date = ''
                        author = ''
                    for item in div.children:
                        if item.name == 'div':
                            item.clear()
                    s = re.sub('[\']+', '`', div.get_text())
                    s = '\'' + re.sub('[\s]+', ' ', s).strip() + '\''
                    if len(s) > 2:
                        theme_messages.append([post_date, author, s])
            # В отдельном цикле ищем ссылку на следующую страницу темы (это 'a' с атрибутом 'rel', равным 'next')
            # Если ссылка найдена, 
            flag = False 
            for link in soup.find_all('a'):
                if not flag and link.has_attr('href') and link.has_attr('rel'):
                    if link.get('rel')[0] == 'next':
                        flag = True
                        s = link.get('href')
                        theme_address = site + 'showthread.php?' + re.search(thread + '[\S]+', s)[0]
        messages.append(['\''+ theme[0] + '\'', theme_messages])
    return messages

In [ ]:
def save_messages(messages, folder_name):
    f_name = os.path.join(folder_name[3], folder_name[3] + '_' + folder_name[0] + '_' + folder_name[1] + '.csv')
    with open(f_name, 'w', encoding='utf-8') as ouf:
        ouf.write('Code,Folder,Theme,Date,Time,Author,Message\n')
        for message in messages:
            header = folder_name[:2] + [message[0]]
            for item in message[1]:
                ouf.write(','.join(header + item))
                ouf.write('\n')

In [ ]:
def scraper(site, folder_names):
    for folder_name in folder_names:
        if len(folder_name[2]) > 0:
            folder = folder_name[2]
            current_themes = collect_themes(site, folder)
            current_messages = collect_messages(site, current_themes)
            save_messages(current_messages, folder_name)
    print('Information collected')

In [ ]:
scraper(site, folder_names)

In [ ]:
#import pandas as pd
#mess = pd.read_csv('./LADA GRANTA/LADA GRANTA_5_Электрооборудование.csv', quotechar="'")
#mess